In [21]:
import pandas as pd
df=pd.read_csv('train/train.csv')

In [22]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [23]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [24]:
df=df.dropna()

In [25]:
X=df.drop('label',axis=1)

In [26]:
y=df['label']

In [27]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
from nltk.stem import WordNetLemmatizer

In [29]:
messages=X.copy()
messages.reset_index(inplace=True)

In [30]:
lr = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [lr.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

In [32]:
type(corpus)

list

In [33]:
len(corpus)

18285

In [34]:
y.shape

(18285,)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=1000,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()

In [36]:
X.shape

(18285, 1000)

In [37]:
#X=X.iloc[:,:].values

In [38]:
X=X.reshape(18285,1000,-1)

In [39]:
X.shape

(18285, 1000, 1)

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [41]:
model = Sequential()
model.add(LSTM(units=100, input_shape = (1000,1) ))
model.add(Dropout(0.2))
model.add(Dense(2,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 41,005
Trainable params: 41,005
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
len(X_train)

12250

In [44]:
len(X_test)

6035

In [45]:
len(y_train)

12250

In [46]:
len(y_test)

6035

In [47]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=256)

Epoch 1/10
39/39 [==============================] - 233s 6s/step - loss: 0.6742 - accuracy: 0.5677 - val_loss: 0.6530 - val_accuracy: 0.5629
Epoch 2/10
39/39 [==============================] - 285s 7s/step - loss: 0.6187 - accuracy: 0.5800 - val_loss: 0.5727 - val_accuracy: 0.5927
Epoch 3/10
39/39 [==============================] - 348s 9s/step - loss: 0.5288 - accuracy: 0.6611 - val_loss: 0.4839 - val_accuracy: 0.7563
Epoch 4/10
39/39 [==============================] - 355s 9s/step - loss: 0.4857 - accuracy: 0.7456 - val_loss: 0.4708 - val_accuracy: 0.7608
Epoch 5/10
39/39 [==============================] - 342s 9s/step - loss: 0.5763 - accuracy: 0.6220 - val_loss: 0.6049 - val_accuracy: 0.5845
Epoch 6/10
39/39 [==============================] - 347s 9s/step - loss: 0.5448 - accuracy: 0.6013 - val_loss: 0.5091 - val_accuracy: 0.6306
Epoch 7/10
39/39 [==============================] - 347s 9s/step - loss: 0.5113 - accuracy: 0.7043 - val_loss: 0.4993 - val_accuracy: 0.7469
Epoch 8/10
39

In [48]:
y_pred1=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [49]:

y_test.shape

(6035,)

In [50]:
y_pred1.shape

(6035, 1)

In [51]:
print(y_pred1)

[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [1]]


In [52]:
y_test.values

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test.values,y_pred1.ravel())

0.7582435791217895

In [54]:
#X_tfidf_train_title.shape

In [55]:
#y_train_title.shape